In [27]:
# Importing important libraries and modules

import re
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize
import nltk
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.naive_bayes import GaussianNB
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import CountVectorizer

In [22]:
# base_url tends to be fixed if there are links in the webpage
base_url = 'https://www.mygov.in/'

# next_page part of the url keeps on changing
next_page = base_url + '/group-issue/share-your-ideas-pm-narendra-modis-mann-ki-baat-26th-march-2017/'

# lists that will store the requisite data
suggestions = []
username = []
date = []

# Maximum number of suggestions we need
max_suggestions = 1000

while next_page and len(suggestions) < max_suggestions : 
    
    # Downloading the webpage
    page = requests.get(next_page)
    soup = BeautifulSoup(page.content, 'html.parser')
    # print(soup.prettify())
    
    # Getting the content of the requisite tags
    s = [elem.get_text() for elem in soup.select('div.comment_body')]
    d = [elem.get_text() for elem in soup.select('span.date_time')]
    u = [elem.get_text() for elem in soup.select('span.username')]
    
    # Filling the lists with the requisite values
    for item in s :
        suggestions.append(item)    
    for item in u :
        username.append(item)
    for item in d :
        date.append(item)

    # Checking for the click of 'View More' button on the webpage    
    next_page = soup.find('li', class_='pager-next first last')
    
    # If the button is clicked, the next_page part of the url changes
    if next_page : 
        next_page = base_url + next_page.find('a').get('href')
        
    # Extra - for testing     
    print('comments: {}'.format(len(suggestions)))


comments: 10
comments: 20
comments: 30
comments: 40
comments: 50
comments: 60
comments: 70
comments: 80
comments: 90
comments: 100
comments: 110
comments: 120
comments: 130
comments: 140
comments: 150
comments: 160
comments: 170
comments: 180
comments: 190
comments: 200
comments: 210
comments: 220
comments: 230
comments: 240
comments: 250
comments: 260
comments: 270
comments: 280
comments: 290
comments: 300
comments: 310
comments: 320
comments: 330
comments: 340
comments: 350
comments: 360
comments: 370
comments: 380
comments: 390
comments: 400
comments: 410
comments: 420
comments: 430
comments: 440
comments: 450
comments: 460
comments: 470
comments: 480
comments: 490
comments: 500
comments: 510
comments: 520
comments: 530
comments: 540
comments: 550
comments: 560
comments: 570
comments: 580
comments: 590
comments: 600
comments: 610
comments: 620
comments: 630
comments: 640
comments: 650
comments: 660
comments: 670
comments: 680
comments: 690
comments: 700
comments: 710
comments: 720
c

In [23]:
# Creating a Dataframe to store the above extracted list values

column_headers = ["Username","Suggestion","Made On","Category"]
df = pd.DataFrame(columns=column_headers)

# Converting the lists into Series objects **Dataframes store Series objects only**

u = pd.Series(username)
d = pd.Series(date)
s = pd.Series(suggestions)

# Inserting the converted lists as columns in the Dataframe

df["Username"] = u.values
df["Suggestion"] = s.values
df["Made On"] = d.values
df

,Username,Suggestion,Made On,Category
0,Hari Mohan Gupta,Sir my humble submission is that please ask pu...,1 month 3 days ago,NaN
1,paras shah,Hello\n Sir.... Mera AK idea hai Jese bus tic...,1 month 3 days ago,NaN
2,Anu Verma,"Respect sir,\nI am Hindi teacher in one of the...",1 month 3 days ago,NaN
3,Jayesh Kulkarni,Sir\nsuggestions AADHAR BASE SYSTEM\n1.Cash Le...,1 month 3 days ago,NaN
4,AKASH GAUTAM,SIR KINDLY LOOK INTO MARITIME SECTOR SPECIALLY...,1 month 3 days ago,NaN
5,Mamata Singh,महोदय आपके सुशासन में सम्बन्धित मंत्रालय के अध...,1 month 3 days ago,NaN
6,Mamata Singh,महोदय यहां सोचने का विषय यह है कि 9 माह के अन्...,1 month 3 days ago,NaN
7,Mamata Singh,महोदय 9 माह का समय बीत गया न जाने इस अवधि में ...,1 month 3 days ago,NaN
8,Mamata Singh,महोदय 9 माह का समय बीत जाने के बाद भी आज तक C...,1 month 3 days ago,NaN
9,Mamata Singh,"आदरणी प्रधानमंत्री जी,\nभारत सरकार\nमहोदय, साद...",1 month 3 days ago,NaN


In [24]:
# Creating dictionary for Category - Keyword Mapping
dict_category = { 'Medical/Safety/Health/Cleanliness' : {"doctor","nurse","medicine","operation","surgery","accident","ambulance","patient"},
                  'Education/Achievement' : {"teacher","book","education","student","school","admission","english","hindi","subject","college","study","maths","science","engineering","bachelors","mba","medal"},
                  'Agriculture/Farming/Energy' : {"electricity","power","solar","thermal","hydro","farm","crop","land"},
                  'Transport' : {"train","railway","car","bus","airplane","plane","traffic","light","road"},  
                  'Employment': {"job","startup","entrepeneur","salary","stipend","employment"},
                  'Govt Initiatives' : {'aadhar','demonitisation'},
                  'Development/Technology': {"machine","youth","technology","digital","research","robot","import","export"},
                  'Finance/Banking': {"tax","money","gst","cheque","profit","loss","bounce","policy","loan"},
                  'Law/Bribery/Crime': {"law","murder","corruption","jail","rape","stab","police","crime"},
                  'Defence/Security' : {"maritime","marine","navy","cadet","army","soldier","border"}}

# List that stores all the categories
category = []

In [25]:
# Function to find the category based on the words in the comment
def find_category(tok_words):
    
    # Instance to stem the words
    ps=PorterStemmer();
    flag = 0
    for item in dict_category.items(): # item = Categories
        for value in item[1]: # value = keywords
            if ps.stem(value) in tok_words:  
                flag = 1
            if flag == 1:
                return (item[0])
    if flag==0:
        return("Others")   # In case there is no match with any of the keywords   
                

In [28]:
# Tokenizing ( Splitting suggestions into lists of words ) suggestions for classification
data = []
all_words = []
category = []
for comment in df["Suggestion"]:
    
    letters_only = re.sub("[^a-zA-Z]", " ", comment) 
    word_list = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))         
    
    ps=PorterStemmer();
    m_words = [ps.stem(w) for w in word_list if not w in stops]   
    
    # Finding the category of the suggestion
    c = find_category(m_words)
    category.append(c)

    # Adding these split words, along with their respective comment's category to 'data'
    l = (m_words,c)
    data.append(l)  
        
    # Creating a list of all words from all suggestions - For frequency calculation    
    for i in m_words:
        # Checks for English words
        all_words.append(i);    
len(all_words)

df["Category"] = pd.Series(category)
print(df)
# The above prints the length of all syntactically correct words from the suggestions we are considering

                             Username  \
0                    Hari Mohan Gupta   
1                          paras shah   
2                           Anu Verma   
3                     Jayesh Kulkarni   
4                        AKASH GAUTAM   
5                        Mamata Singh   
6                        Mamata Singh   
7                        Mamata Singh   
8                        Mamata Singh   
9                        Mamata Singh   
10                       AKASH GAUTAM   
11                       Hafijul Khan   
12                Goverdhan Thirumala   
13                 Deepikaah S Tejaal   
14                         Suruchi_17   
15                         Suruchi_17   
16                       Hafijul Khan   
17                         Suruchi_17   
18                   TAPAN RANJAN DAS   
19                       Hafijul Khan   
20                       Pawan Mishra   
21                       Pawan Mishra   
22                      U g Sreekanth   
23              

In [29]:
# Calculates the frequency of the words
all_words = nltk.FreqDist(all_words)
#all_words
print(len(all_words.most_common()))

5944


In [30]:
# Picking the most occuring words
word_features = list(all_words.keys())[:3000]

In [31]:
# Just like in array, where 0s and 1s marked the absence and presence of words
# here, false and true carry out the same functionality
# In that case, we had made an array, here its a dictionary
def find_features(data):
    words = set(data)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [32]:
# Generating the feature set as before
featuresets = [(find_features(comment), category) for (comment, category) in data]

In [33]:
# Training set
training_set = featuresets[:750]

# Testing set
testing_set = featuresets[750:]

In [34]:
# Various Classifiers

## Naive Bayes Classifiers
NB_classifier = nltk.classify.NaiveBayesClassifier.train(training_set)
print("NaiveBayes accuracy percent:",nltk.classify.accuracy(NB_classifier, testing_set)*100)

classifier = nltk.NaiveBayesClassifier.train(training_set)
print("NaiveBayes accuracy percent:",nltk.classify.accuracy(classifier, testing_set)*100)

MNB_classifier = SklearnClassifier(MultinomialNB()).train(training_set)
print("MultinomialNB accuracy percent:",nltk.classify.accuracy(MNB_classifier, testing_set)*100)

BNB_classifier = SklearnClassifier(BernoulliNB()).train(training_set)
print("BernoulliNB accuracy percent:",nltk.classify.accuracy(BNB_classifier, testing_set)*100)


## Random Forest Classifier
#RF_classifier = RandomForestClassifier(n_estimators=100).fit(df["Suggestion"],df["Category"])
#print("RandomForest accuracy percent",nltk.classify.accuracy(RF_classifier,testing_set)*100)

#LogisticRegression_classifier = SklearnClassifier(LogisticRegression()).train(training_set)
#print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

#SGDClassifier_classifier = SklearnClassifier(SGDClassifier()).train(training_set)
#print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)


## SVM Classifiers
SVC_classifier = SklearnClassifier(SVC()).train(training_set)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC()).train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)



('NaiveBayes accuracy percent:', 50.4)
('NaiveBayes accuracy percent:', 50.4)
('MultinomialNB accuracy percent:', 52.800000000000004)
('BernoulliNB accuracy percent:', 42.4)
('SVC_classifier accuracy percent:', 40.400000000000006)
('LinearSVC_classifier accuracy percent:', 53.6)


In [35]:
# CLUSTERING 
# Unable to implement Clustering because our feature set is a dictionary and not an array
# The following function call takes in an array only
# kmeans = KMeans(n_clusters=10, random_state=0).fit(training_set)